In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

# Generate simple sequential data (e.g., stock prices)
X = np.array([[i, i+1, i+2] for i in range(100)])
y = np.array([i+3 for i in range(100)])

# Reshape X for RNN (samples, timesteps, features)
X = X.reshape((X.shape[0], X.shape[1], 1))

# Build the RNN model
model = Sequential([
    SimpleRNN(10, activation="tanh", input_shape=(3, 1)),
    Dense(1)  # Output layer for regression
])

# Compile the model
model.compile(optimizer="adam", loss="mse")

# Train the model
model.fit(X, y, epochs=10, verbose=1)

# Make predictions
y_pred = model.predict(X[:5])
print("Predictions:", y_pred.flatten())


C:\Users\ADMIN\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 3511.7249
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 3799.6477
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 3595.0337 
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 3569.0178
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 3558.1914
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 3325.6567
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 3592.2744
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 3431.6692
Epoch 9/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 3466.5625
Epoch 10/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 3533.6787 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
Predictions: [1.2075286 1.4513122 1.350757  1.2636034 1.2161499]


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# 📌 Sample text data
text = """The sun rises in the east and sets in the west.
          The moon shines at night.
          The stars twinkle in the sky.
          The earth revolves around the sun.
          The universe is vast and mysterious."""

# 1️⃣ Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])  # Learn the vocabulary
total_words = len(tokenizer.word_index) + 1  # Add 1 for padding

# 2️⃣ Create input sequences
input_sequences = []
words = text.split()
for i in range(1, len(words)):
    n_gram_sequence = words[:i+1]  # Create word sequences
    input_sequences.append(tokenizer.texts_to_sequences([" ".join(n_gram_sequence)])[0])

# 3️⃣ Pad sequences for uniform length
max_seq_length = max(len(seq) for seq in input_sequences)
input_sequences = pad_sequences(input_sequences, maxlen=max_seq_length, padding="pre")

# 4️⃣ Split into features (X) and target (y)
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

# 5️⃣ Build the LSTM-based RNN model
model = Sequential([
    Embedding(total_words, 10, input_length=max_seq_length-1),
    LSTM(50, return_sequences=True),
    LSTM(50),
    Dense(total_words, activation="softmax")
])

# 6️⃣ Compile the model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# 7️⃣ Train the model
model.fit(X, y, epochs=50, verbose=1)

# 8️⃣ Function to generate text
def generate_text(seed_text, next_words=5):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_seq_length-1, padding="pre")
        predicted = np.argmax(model.predict(token_list), axis=-1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

# 🔥 Test the text generator
print(generate_text("The sun rises", next_words=5))


Epoch 1/50


C:\Users\ADMIN\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - accuracy: 0.0306 - loss: 3.1351
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.2449 - loss: 3.1295
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.2449 - loss: 3.1258
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.2449 - loss: 3.1221
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.2449 - loss: 3.1185
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.2345 - loss: 3.1144
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.2449 - loss: 3.1068
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.2449 - loss: 3.0963
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.2345 - loss: 3.0868
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.2345 - loss: 3.0721
Epoch 11/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.2449 - loss: 3.0453
Epoch 12/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.2449 - loss: 3.0171
Epoch 13/50


In [3]:
print(generate_text("The moon shines", next_words=5))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
The moon shines in in in in in
